# **최적의 검색 방식를 위한 테스트 파일**

1. 목적: 어떤 Retriever 유형이 최종 답변의 정확도에 더 기여하는지 분석

2. 실험 조건

    > Dense 단독 (ex. OpenAIEmbedding, HuggingFace 등)

    > Ensemble 조합 (가중치 비율 조정: dense 0.7 / sparse 0.3 등)

    > 파라미터 조정: top_k


## **환경설정(실험동안 변경 X)**
- DB 연결
- 모델 설정
- 프롬프트 설정
- 임베딩 모델 설정
- 벡터스토어 설정

In [15]:
# 테스트를 위한 셋업
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드 (.env)
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Pinecone 인덱스 설정
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "hufs-chatbot"

# 모델 설정
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini"  
)

# 프롬프트 정의
# 문서 간 정보를 **종합해서** 답변하는 것이 중요합니다.
prompt = PromptTemplate.from_template("""
너는 한국외대 학사 관련 질문에 답변하는 챗봇이야.
다음은 검색된 문서 내용이야. 이 내용을 바탕으로 사용자의 질문에 답변해.
답을 모르면 모른다고 해. 무조건 한국어로 높임말을 사용해서 답변해.

#질문:
{question}

#문서 내용:
{context}

#답변:
""")

# 임베딩 모델, 벡터스토어 설정
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")  

vectorstore = PineconeVectorStore.from_existing_index(
    index_name="hufs-chatbot",
    embedding=embedding,
     text_key="page_content" 
)

In [16]:
# sparse retriever test를 위한 doc_list 생성
from loader.load_documents import *
# 1. 경로 설정
timetable_path = "data/Timetable_Crawling_Data"
guidebook_path = "data/major_guide_2025.pdf"
pram_path = "data/pram_2025_1.json"
prof_path = "data/hufs_professor.json"
college_path = "data/hufs_colleges.json"
notice_path = "data/hufs_notice.json"
schedule_path = "data/hufs_schedule.json"

# 2. 문서 로드
print("강의시간표 로드 중...")
subject_docs = load_all_subject_documents(timetable_path)

print("전공가이드북 로드 중...")
guidebook_docs = load_major_guidebook(guidebook_path)

print("수강편람 로드 중...")
pram_docs = load_sugang_pram(pram_path)

print("교수진 정보 로드 중...")
prof_docs = load_professor_documents(prof_path)

print("단과대 정보 로드 중...")
college_docs = load_college_intro(college_path)

print("학사공지 로드 중...")
notice_docs = load_notice_documents(notice_path)

print("학사일정 로드 중...")
schedule_docs = load_academic_schedule(schedule_path)

doc_list = subject_docs + guidebook_docs + pram_docs + prof_docs + college_docs + notice_docs +schedule_docs

강의시간표 로드 중...
전공가이드북 로드 중...
수강편람 로드 중...
교수진 정보 로드 중...
단과대 정보 로드 중...
학사공지 로드 중...
학사일정 로드 중...


## **검색기 설정**
- 검색기 구조를 변경해가며 최적의 성능을 내는 검색기 확인
- 각 내용, 조합별 성능이나 문제점 등 기록 필수

In [17]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# doc_list가 Document 객체 리스트일 경우, text만 추출
texts = [doc.page_content for doc in doc_list]

# bm25 retriever 초기화
bm25_retriever = BM25Retriever.from_texts(texts)
bm25_retriever.k = 20  # BM25Retriever의 검색 결과 개수

# 실험에 따라 변경될 내용
pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, pinecone_retriever],
    weights=[0.3, 0.7]
)

### 검색 문서 수
- 초기설정: 5개 -> 검색된 문서의 수가 적어 원하는 정답문서를 포함하지 않는 케이스가 많음
- 1차 수정: 10개 -> 정답문서가 포함이 안되거나, 포함되어도 해당 문서를 기반으로 답변하지 않는 케이스 존재
- 2차 수정: 20개 -> 프롬프트 입력토큰 수 제한에 걸림 -> 4o-mini로 설정 -> 정답문서가 포함되는 케이스가 많고 해당 정답문서 기반 답변
- taken: 검색해오는 문서의 개수가 늘어나면 정답문서가 포함될 가능성이 높아짐 -> parent-document 리트리버가 불필요할 수 있음

### 검색 방식
> 앙상블 리트리버 초기 가중치 설정: [0.5,0.5] 
- Sparse retriever의 검색 결과가 생각보다 별로 좋지 않음. 실제로 관련이 있는 문서를 가져오는 케이스가 10개 중 2개 정도 -> **sparse의 가중치를 낮추기로 결정**
> 앙상블 리트리버 가중치 1차 수정: [0.3, 0.7]
- 검색되는 문서에 실제로 관련있는 문서들이 포함되는 정도가 개선되었음

## **체인**
- 변경 X 순서도 retriever 설정 뒤로 오도록 할 것

In [18]:
# 체인 구성
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## **실험**
- 질문은 기존에 문제를 갖고 있던 질문과 선정된 질문 들로 구성
- 답변 결과 해석: 정확히 답변 했는지, 정확한 답변이 아니라면 어떤 문제가 있었고 어떻게 개선해야 하는지

In [17]:
# 질문 - 답변 확인
query1 = "후기 이중이 뭐야?" # 질문 입력 - 올해 신청일 언제였는지 알아?
ensemble_result = chain.invoke(query1)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

후기 이중전공은 부전공을 이수 중인 학생이 정규 8학기 내에 제1전공 졸업학점(부전공 21학점 포함)을 모두 취득한 후, 추가로 1~2학기를 더 수학하여 해당 부전공의 이중전공화에 필요한 나머지 학점을 취득함으로써 제1전공과 이중전공 학위를 동시에 취득할 수 있는 제도입니다. 

신청은 매년 4월에 한 번만 가능하며, 신청 자격은 현재 7~8학기 이수 중인 학생으로, 부전공을 이수 중인 경우에 해당합니다. 추가학기를 등록해야 하며, 등록금은 후기 이중전공 학과의 등록금으로 납부해야 합니다. 

더 궁금하신 점이 있으시면 말씀해 주시기 바랍니다.


- 앙상블리트리버, 문서 20개 검색 기반 답변
- 입력 토큰 제한에 걸려서 gpt-4o-mini 모델 사용
- 검색된 문서 내 정답 문서 존재, 정답 문서기반 정확한 답변 => 7번째 문서에 포함되어 있었음.
- '제도'에 대한 정확한 답변 출력
- '올해 신청일 언제였는지 알아?'와 같은 질문을 통해 multi-turn chain 검증 가능

In [22]:
# 질문 - 답변 확인
query12 = "일본어통변역학과 FLEX 시험 점수 제출 기간 알고 싶어."
ensemble_result = chain.invoke(query12)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

일본어통번역학과의 FLEX 시험 점수 제출 기간은 2025년 4월 16일(수)부터 4월 23일(수)까지입니다. 이 기간 내에 해당 학과 사무실에 신청하셔야 하며, 기한 이후에는 추가 신청이 불가하니 유의하시기 바랍니다. 추가적인 문의사항이 있으시면 학과 사무실에 직접 문의하시기 바랍니다.


In [43]:
# 질문 - 답변 확인
query2 = "글로벌스포츠산업학부 졸업하면 뭐해?" # 질문 입력 - 올해 신청일 언제였는지 
ensemble_result = chain.invoke(query2)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

글로벌스포츠산업학부를 졸업하신 후에는 다양한 진로를 선택하실 수 있습니다. 주로 다음과 같은 분야에서 활동하게 됩니다:

1. **스포츠 매니지먼트 및 이벤트 기획**: 스포츠 클럽, 리그, 대회, 이벤트 등을 기획하고 운영하는 직무입니다.
2. **스포츠 마케팅 및 홍보**: 스포츠 산업에서 제품, 서비스, 이벤트 등을 마케팅하고 홍보하는 업무를 수행합니다.
3. **스포츠 데이터 분석가**: 스포츠 경기에서 발생하는 데이터를 수집하고 분석하여 팀이나 선수의 성과를 향상시키는 데 필요한 정보를 도출하는 직무입니다.
4. **스포츠 미디어**: 스포츠 뉴스, 인터뷰, 해설, 하이라이트 등 다양한 스포츠 관련 콘텐츠를 제작하고 편집하여 방송, 인터넷, 모바일 등에서 전달하는 업무를 맡습니다.

이 외에도 졸업 후에는 대학원 진학이나 관련 분야의 연구를 진행할 수 있는 기회도 있습니다.


- 위 질문과 사용 조건 동일
- 검색된 문서 내 정답 문서 존재, 정답 문서기반 정확한 답변 => 14번째 문서에 포함되어 있었음.
- 전공 가이드북 청크의 졸업후 진로 관련 내용을 정확히 가져옴

In [5]:
# 질문 - 답변 확인
query3 = "감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?" 
ensemble_result = chain.invoke(query3)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조건이 있습니다. 일반적으로 의원에서 발급받은 서류는 유고결석 인정에 해당되지 않으며, 병원급 이상의 의료기관에서 발급된 증빙서류가 필요합니다. 따라서, 감기로 병원급 의료기관에서 진료를 받으셨다면 해당 증빙서류를 제출하시면 유고결석이 가능할 수 있습니다.

정확한 사항은 담당 교수님께 문의하시고, 필요한 서류를 제출하시는 것이 좋습니다. 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.


- 정확한 답변
- gpt-4o모델의 추론성능이 뛰어나서 정확하게 답변을 하는 것으로 보임
- 검색해온 문서도 기존의 dense retriever만 활용했을 때 보다 연관성 있는 문서들을 훨씬 더 잘 가져옴

In [14]:
# 질문 - 답변 확인
query4 = "국제스포츠산업론 강의하는 교수님 이름이 뭐였지? 연락처를 알고싶은데" 
ensemble_result = chain.invoke(query4)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

국제스포츠산업론 강의를 담당하시는 교수님은 김남수 교수님이십니다. 교수님의 이메일 주소는 arpkim@gmail.com이며, 연구실은 어문학관 524호입니다. 연락처는 031-330-4340입니다. 추가로 궁금하신 점이 있으시면 말씀해 주시기 바랍니다.


- 부정확한 답변
- 강의시간표에 있는 내용까지 같이 확인하고 답변해야되는데 교수진정보의 정보만을 기반으로 답변함
- 프롬프트 설정에서 여러 문서를 종합적으로 고려해서 답변해야되는 케이스에 대한 지시사항이나 few-shot 프롬프팅, CoT 프롬프팅 수행해야함

In [28]:
# 질문 - 답변 확인
query5 = "25년도 1학기 금요일 1교시 스포츠경영인턴십 강의하시는 교수님 성함이랑 연락처 알 수 있나?" 
ensemble_result = chain.invoke(query5)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

죄송하지만, 2025년도 1학기 금요일 1교시에 개설된 "스포츠경영인턴십" 강의의 교수님 성함과 연락처에 대한 정보는 제공되지 않았습니다. 다른 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.


- 부정확한 답변
- 문제 상황 가설 1: 강의시간표 문서의 현재 형태를 이해하지 못한다
- 문제 상황 가설 2: 다중 문서 해석에 대응하지 못한다
- 프롬프트 엔지니어링, 문서 재정립으로 대응

In [33]:
# 질문 - 답변 확인
query6 = "면학장학금 추가지급도 해?" 
ensemble_result = chain.invoke(query6)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

면학장학금의 추가 지급에 대한 정보는 문서 내용에 포함되어 있지 않습니다. 따라서 해당 질문에 대한 정확한 답변을 드리기 어렵습니다. 추가적인 정보가 필요하시다면 한국외대의 공식 웹사이트나 관련 부서에 문의하시기를 권장드립니다.


In [5]:
# 질문 - 답변 확인
query7 = "기후변화융합학부 외국어인증 필요해? 토익 기준으로 몇점이어야돼?" 
ensemble_result = chain.invoke(query7)
#bm25_result = bm25_retriever.invoke(query)
#pinecone_result = pinecone_retriever.invoke(query)
print(ensemble_result)

기후변화융합학부에서는 외국어 인증이 필요합니다. 토익 기준으로는 700점 이상을 요구하고 있습니다. 추가로 궁금하신 점이 있으시면 언제든지 말씀해 주시기 바랍니다.


## **검색된 문서 확인**

In [18]:
# query1 = "후기 이중이 뭐야?"
docs = ensemble_retriever.invoke(query1)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)



--- 문서 1 ---
2025년 후기 학위수여식 일정:
2025-08-22 (금요일)
메타데이터: {}

--- 문서 2 ---
[공통] 후기이중전공 신청 공고

[공통] 후기이중전공 신청 공고

(공통) 후기이중전공 신청 공고

2025학년도 후기이중전공 신청 일정 및 방법 등을 공고합니다.

1. 신청기간: 2025 4. 14.(월) 10:00 ~ 4. 18.(금) 16:00까지

2. 신청대상: 현재 7~8학기 이수중인 자로, 후기이중전공을 희망하는 전공을 부전공으로 이수중인 자

(7학기를 마치고 휴학한 학생도 이번 신청기간에 신청할 것)

3. 신청방법: 인터넷 종합정보시스템에서 신청 (로그인 > 전공/교직관리 > 후기이중전공신청)

4. 허가자 발표: 7월초 홈페이지 공고(예정)

5. 기타사항

가. 1년에 1회, 4월에만 신청하므로 반드시 희망자는 이번 신청 일정을 따를 것

나. 지원학과 입학정원의 50 ~ 150% 이내에서 학과 자율 선발하되, 학과에서 요구하는 기본 자격요건 미충족 시 탈락될 수 있음

(세부 배정기준은 지원학과 학과장실 문의 또는 학과 홈페이지 참조)

다. 이수자 선발 이후 8학기까지 전공심화(+부전공) 또는 일반 부전공에 맞는 졸업에 필요한 졸업학점 요건을 모두 완료하여야 최종적으로 허가

라. 조기졸업대상자, 학석사연계과정/석사예약제 이수자는 신청 불가(신청 희망 시 내방하여 해당 자격 취소처리 후 신청)

마. 후기이중전공으로 선발되면 추가학기를 1학기 이상 무조건 등록, 수강해야 하며 등록금은 후기이중전공 학과의 등록금으로 납부해야 함

바. 기타 자세한 사항은 전자규정집 "이중전공시행규정" 참조

2025. 4.

교무처장

| 0 |
|:------------------------------|
| (공통) 후기이중전공 신청 공고 |
메타데이터: {'author': '학사종합지원센터', 'date': '2025.04.04', 'doc_type': '학사공지', 'ntt_id': '223498', 'title'

In [45]:
# query2 = "글로벌스포츠산업학부 졸업하면 뭐해?" 
docs = ensemble_retriever.invoke(query2)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)



--- 문서 1 ---
이승구 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. 연구분야는 스포츠미디어, 스포츠방송저널리즘입니다. 이메일은 natioaldocu@daum.net입니다.
메타데이터: {}

--- 문서 2 ---
박지훈 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Seoul National University, English literature 학위를 가지고 있습니다. 연구분야는 스포츠법입니다. 이메일은 powerpitcher61@gmail.com입니다.
메타데이터: {}

--- 문서 3 ---
서형석 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Seoul National University (BA), Hankuk Univ. of Foreign Studies (MS) 학위를 가지고 있습니다. 연구분야는 E스포츠입니다. 이메일은 esports@gmail.com입니다.
메타데이터: {}

--- 문서 4 ---
김상철 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Ph.D Michigan State University, Engineering 학위를 가지고 있습니다. 연구분야는 스포츠컴퓨터프로그래밍, 스포츠컴퓨터프로그래밍언어입니다. 이메일은 kimsa@hufs.ac.kr입니다. 연구실은 공학관 416호입니다.
메타데이터: {}

--- 문서 5 ---
김명락 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Master of University of Helsinki, Global Management-IT 학위를 가지고 있습니다. 연구분야는 스포트데이터분석과R실습, 스포츠빅데이터분석입니다. 이메일은 endbase@naver.com입니다.
메타데이터: {}

--- 문서 6 ---
Culture&Technology융합대학

4차 산업혁명이 촉발한 변화의 물결 속에서 고등교육 패러다임의 근

In [6]:
# query3 = "감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?" 
docs = ensemble_retriever.invoke(query3)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)



--- 문서 1 ---
| 질병 및 상해 로 인한 입원 또는 감염병 으로 인한 격 리(치료) | 법정감염병 (**질병관리청의 법정감염병에 한함) | 격리(치료) 기간 또는 감염우려 기간 | 해당 증빙서류 | 담당교수에게 증빙서류 제출 |  |
| 졸업시험 | 졸업시험 | 졸업시험 당일 | 학과장 확인서 | 담당교수에게 증빙서류 제출 |  |
| 수업일수 4분의 3선 후 징병에 의한 군입대 | 수업일수 4분의 3선 후 징병에 의한 군입대 | - | 입영통지서 | 담당교수에게 증빙서류 제출 |  |
| 출산 | 출산 | 본인:20일 배우자:10일 | 본인 : 출생(출산예정)증 명서 등, 배우자 : 출생 (출산예정)증명서 및 가족관계증명서 등 | 담당교수에게 증빙서류 제출 |  |
| 생리 | 생리 | 생리기간 (학기중 2회) | 유고결석신청서 (생리공결제) | 담당교수에게 증빙서류 제출 | ※ 자세한 사항은 학사공지사항 생리 유고 신청안내 참조 |
| 교직과정이수자의 학교현장실습 | 교직과정이수자의 학교현장실습 | 교육실습 기간 | 교직담당자가 발급한 확인서 | 담당교수에게 증빙서류 제출 |  |
| 졸업예정자(해당학기 졸업가능자) 로서 조기취업자 | 졸업예정자(해당학기 졸업가능자) 로서 조기취업자 | 취업기간 | 학과장 승인받은 유고결석 인정원, 재직(계약)증명서 및 4대보험가입내역 확인서 또는 이에 준하는 재직증명서류 | 입사확정 후 즉시 ￬ 유고결석인정원과 증빙서류에 대해 학과장 승인 ￬ 소속 단과대에 제출 ￬ 유고결석확인서를 발급받아 담당교수에게 제출 ￬ 수업종료 직전 취업사실에 대해 재확인 | ※ 자세한 사항은 학사공지사항 졸업예정자(해 당학기 졸업가능자)의 조기취업 유고결석 신청 안내 참조 |
메타데이터: {}

--- 문서 2 ---
## [paragraph] ##
나. 처리절차

## [paragraph] ##
1) 유고결석·결시 공통 안내사항

## [paragraph] ##
* 질병 및 상해의 경우 의원에서 발급받은 서류는 해당되지 않음
(병원

In [16]:
# query4 = "국제스포츠산업론 강의하는 교수님 이름이 뭐였지? 연락처를 알고싶은데" 
docs = ensemble_retriever.invoke(query4)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)



--- 문서 1 ---
이승구 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. 연구분야는 스포츠미디어, 스포츠방송저널리즘입니다. 이메일은 natioaldocu@daum.net입니다.
메타데이터: {'doc_type': '교수진 정보', '연구분야': '스포츠미디어, 스포츠방송저널리즘', '연구실': '정보없음', '이름': '이승구 교수', '이메일': 'natioaldocu@daum.net', '전화번호': '정보없음', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': '정보없음'}

--- 문서 2 ---
서형석 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Seoul National University (BA), Hankuk Univ. of Foreign Studies (MS) 학위를 가지고 있습니다. 연구분야는 E스포츠입니다. 이메일은 esports@gmail.com입니다.
메타데이터: {'doc_type': '교수진 정보', '연구분야': 'E스포츠', '연구실': '정보없음', '이름': '서형석 교수', '이메일': 'esports@gmail.com', '전화번호': '010-3129-0474', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': 'Seoul National University (BA), Hankuk Univ. of Foreign Studies (MS)'}

--- 문서 3 ---
김상철 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Ph.D Michigan State University, Engineering 학위를 가지고 있습니다. 연구분야는 스포츠컴퓨터프로그래밍, 스포츠컴퓨터프로그래밍언어입니다. 이메일은 kimsa@hufs.ac.kr입니다. 연구실은 공학관 416호입니다.
메타데이터: {'doc_type': '교수진 정보', '연구분야': '스포츠컴퓨터프로그래밍, 스포츠컴퓨터프로그래밍언어'

In [29]:
# query5 = "25년도 1학기 금요일 1교시 스포츠경영인턴십 강의하시는 교수님 성함이랑 연락처 알 수 있나?" 
docs = ensemble_retriever.invoke(query5)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)


--- 문서 1 ---
[스포츠ESG파트너십] 2학년 대상 / 교수: 이승필 / 강의시간: 수요일 ,4,5,6교시 / 강의실 0301호 / 학점: 3 / 전공필수 과목 맞음 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님

[스포츠경영연구법] 2학년 대상 / 교수: 김남수 / 강의시간: 목요일 ,1,2,3교시 / 강의실 2301호 / 학점: 3 / 전공필수 과목 맞음 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님

[스포츠마케팅실무실습] 2학년 대상 / 교수: 박성희 / 강의시간: 화요일 ,7,8,9교시 / 강의실 0209호 / 학점: 3 / 전공필수 과목 맞음 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님

[스포츠미디어기획및창착실습(캡스톤)] 3학년 대상 / 교수: 이승구 / 강의시간: 월요일 ,4,5,6교시 / 강의실 0212호 / 학점: 3 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 맞음 / 원어강의 아님

[스포츠방송저널리즘실습] 3학년 대상 / 교수: 이승구 / 강의시간: 월요일 ,7,8,9교시 / 강의실 0212호 / 학점: 3 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 맞음 / 원어강의 아님
메타데이터: {'doc_type': '강의시간표', 'filename': '1학기전공_과목정보(글로벌).csv', '개설영역': '(글로벌) - 글로벌스포츠산업전공'}

--- 문서 2 ---
[글로벌e스포츠산업론] 2학년 대상 / 교수: 서형석 / 강의시간: 화요일 ,4,5,6교시 / 강의실 1511호 / 학점: 3 / 전공필수 과목 맞음 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님

[글로벌e스포츠컨텐츠기획및실습] 3학년 대상 / 교수: 류윤지 / 강의시간: 화요일 ,7,8,9교시 / 강의실 1511호 / 학점: 3 / 전공필수 과목 아님 / 온라인강의 아님 / P/F 평가 과목 아님 / 원어강의 아님

[글로벌e스포츠시설관리론] 4학년 대

In [6]:
# query7 = "기후변화융합학부 외국어인증 필요해? 토익 기준으로 몇점이어야돼?" 
docs = ensemble_retriever.invoke(query7)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)


--- 문서 1 ---
기후변화융합학부

기후변화융합학부는 기후변화 대응 패러다임 변화에 대응할 수 있는 교육과정을 마련하고 국내외 유관기관과의 협력강화를 통해 기후변화 관련 전문지식을 갖춘 인재양성을 도모하고자 신설되었습니다. 이를 위해 에너지와 기후변화 정책 및 관련 국제협력 분야에서 기후변화 대응산업과 연구를 선도할 융합인력 양성이 목표이며 기후감시 및 예측, 기후변화 정책, 온실가스 및 에너지 관리 3개 트랙으로 교육과정을 운영할 예정입니다. 이와 같은 교육과정에 따라 사회과학적 소양과 자연과학/공학과의 융복합적인 교육을 통해 과학 및 정책을 포함한 미래의 기후변화 관련 업무를 국제사회에서 선도할 수 있는 거시적 안목을 갖춘 글로벌 융합인재를 배출하고자 합니다.

031-330-4114
메타데이터: {}

--- 문서 2 ---
[공통] 외국어인증 신청서 제출 안내

031) 330-4457

자연과학대학 / 공과대학 / 바이오메디컬공학부

yyj1730@hufs.ac.kr

031) 330-4073

경상대학 / AI융합대학(글로벌) / Culture& Technology융합대학 / 기후변화융합학부

minsun@hufs.ac.kr

031) 330-4953

교무처 학사종합지원센터

| 0 |
|:----------------------------|
| 외국어인증 신청서 제출 안내 |
메타데이터: {}

--- 문서 3 ---
학자를 배출함으로써, 국제사회에서 탄소외교의 신뢰성을 확보할 수 
있을 것으로 기대합니다. 
-  기후변화 관련 국제적 수준의 연구경험을 쌓을 수 있는 기회를 제공
합니다.
-  대기환경기사 또는 온실가스 관리기사 자격증을 취득할 수 있는 커리
큘럼으로 구성되어 있습니다.
주요 전공과목 
기후변화융합학부는 온실가스 감시 및 예측, 기후변화경제, 에너지 관
리 및 정책 총 세 개의 트랙으로 구성되어 있습니다. 각 트랙별로 짜임
새 있는 강의 계획을 통해, 해당 트랙에 관심이 있는 학부생들에게 학습 
방향을 제시해줄 것입니다.


In [23]:
docs = ensemble_retriever.invoke(query12)

# 출력
for i, doc in enumerate(docs):
    print(f"\n--- 문서 {i+1} ---")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)


--- 문서 1 ---
[공통] 2025-1학기(상반기) 교내 FLEX 졸업시험 시행 안내

[공통] 2025-1학기(상반기) 교내 FLEX 졸업시험 시행 안내

[ 공통 ] 2025-1 학기 ( 상반기 ) 교내 FLEX 졸업시험 시행 안내

2025 학년도 1 학기 ( 상반기 ) 교내 FLEX 졸업시험의 세부 시행 계획을 아래와 같이 안내합니다 .

1. 시 험 명 : 2025 학년도 1 학기 (2025 년 상반기 ) 교내 FLEX 졸업시험

2. 시행주관 : FLEX 센터

3. 대상언어 : 총 15 개 언어

※ 포르투갈어 , 네덜란드어 , 스웨덴어 , 마 ・ 인어 , 태국어 , 튀르키예어 ( 터키어 ), 이탈리아어 , 체코어 , 헝가리어 , 세르비아어 , 루마니아어 , 폴란드어 , 그리스어 , 불가리아어 , 힌디어

4. 응시자격 : 교내 각 학과에 사전 신청한 학생만 응시 가능

5. 시험일시 : 2025. 5. 24( 토 ) 10:00~12:00(120 분 )

6. 입실시간 : 09:40 분까지 해당 고사실 내 입실완료 , 마스크 착용 권고

※ 응시자는 정해진 입실시간을 반드시 준수하여야 합니다 .

※ 듣기시험이 시작되면 고사실 내 입실을 할 수 없습니다 .

7. 시험장소 : 본교 서울캠퍼스 교수학습개발원

※ 시험 당일 건물 앞에 설치되는 배치표 , 유도표시 및 붙임의 시행계획 참고

8. 응시자 준비물 ( 반드시 지참 )

※ 신분증 ( 학생증 , 주민등록증 , 운전면허증 , 여권 등 ), 컴퓨터용 수성싸인펜 , 수정테이프

※ OMR 답안 수정은 응시자가 수정테이프로 자유롭게 수정이 가능함 .

9. 시험 및 평가유형 ( 붙임의 시행계획서 참고 )

가 . FLEX 듣기 · 읽기 A 형 ( 듣기 50 문항 , 읽기 90 문항 / 총 120 분 )

나 . FLEX 듣기 · 읽기 B 형 ( 듣기 55 문항 , 읽기 95 문항 / 총 120 분 )

다 . 언어별로 각각 개발되어 있는 유형에 따라 시행

10. 기타사항

가 . 별도 성적

## 2-stage RAG
- 검색 단계를 2단계로 구조화
- 1단계: Bi-Encoder를 이용한 빠른 검색(ensemble retriever)
- 2단계: reranker를 활용한 검색된 문서 순위화 및 compressor를 이용한 압축

# 불필요

In [39]:
'''
# 가져온 문서를 출력
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"metadata: \n {doc.metadata}")
    print(f"Content: \n {doc.page_content}")
    print()
print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"metadata: \n {doc.metadata}")
    print(f"Content:\n {doc.page_content}")
    print()

print("[pinecone Retriever]")
for doc in pinecone_result:
    print(f"metadata: \n {doc.metadata}")
    print(f"Content:\n {doc.page_content}")
    print()
'''

'\n# 가져온 문서를 출력\nprint("[Ensemble Retriever]")\nfor doc in ensemble_result:\n    print(f"metadata: \n {doc.metadata}")\n    print(f"Content: \n {doc.page_content}")\n    print()\nprint("[BM25 Retriever]")\nfor doc in bm25_result:\n    print(f"metadata: \n {doc.metadata}")\n    print(f"Content:\n {doc.page_content}")\n    print()\n\nprint("[pinecone Retriever]")\nfor doc in pinecone_result:\n    print(f"metadata: \n {doc.metadata}")\n    print(f"Content:\n {doc.page_content}")\n    print()\n'

In [5]:
# 검색된 문서 확인
docs = vectorstore.similarity_search(question, k=5) # 검색 문서 수는 검색기에서 설정한 문서 수와 통일

for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content)  # 문서 내용 미리보기

{'author': '학사종합지원센터', 'date': '2025.04.04', 'doc_type': '학사공지', 'ntt_id': '223498', 'title': '[공통] 후기이중전공 신청 공고', 'url': 'https://www.hufs.ac.kr/bbs/hufs/2181/223498/artclView.do'}
[공통] 후기이중전공 신청 공고

[공통] 후기이중전공 신청 공고

(공통) 후기이중전공 신청 공고

2025학년도 후기이중전공 신청 일정 및 방법 등을 공고합니다.

1. 신청기간: 2025 4. 14.(월) 10:00 ~ 4. 18.(금) 16:00까지

2. 신청대상: 현재 7~8학기 이수중인 자로, 후기이중전공을 희망하는 전공을 부전공으로 이수중인 자

(7학기를 마치고 휴학한 학생도 이번 신청기간에 신청할 것)

3. 신청방법: 인터넷 종합정보시스템에서 신청 (로그인 > 전공/교직관리 > 후기이중전공신청)

4. 허가자 발표: 7월초 홈페이지 공고(예정)

5. 기타사항

가. 1년에 1회, 4월에만 신청하므로 반드시 희망자는 이번 신청 일정을 따를 것

나. 지원학과 입학정원의 50 ~ 150% 이내에서 학과 자율 선발하되, 학과에서 요구하는 기본 자격요건 미충족 시 탈락될 수 있음

(세부 배정기준은 지원학과 학과장실 문의 또는 학과 홈페이지 참조)

다. 이수자 선발 이후 8학기까지 전공심화(+부전공) 또는 일반 부전공에 맞는 졸업에 필요한 졸업학점 요건을 모두 완료하여야 최종적으로 허가

라. 조기졸업대상자, 학석사연계과정/석사예약제 이수자는 신청 불가(신청 희망 시 내방하여 해당 자격 취소처리 후 신청)

마. 후기이중전공으로 선발되면 추가학기를 1학기 이상 무조건 등록, 수강해야 하며 등록금은 후기이중전공 학과의 등록금으로 납부해야 함

바. 기타 자세한 사항은 전자규정집 "이중전공시행규정" 참조

2025. 4.

교무처장

| 0 |
|:------------------------------|
| (공통) 후기이중전공